# Taguchi Array

In [2]:
import numpy as np
import pandas as pd
from tabulate import tabulate
from utils import validate_parameters, score_vals, efficiency_calc

In [3]:
MODULE = 0.8e-3
MC = 0.8467e-3

## Input parameter values
#### Currently assumes we will use values from the paper (added to the dict seperately)

In [4]:
# Designate target parameter values
# in_vals = [z_s, z_r2, x_s, x_r2, Cl, gear_thickness] # Cl_i = clearance (i = 1,2,3)
param_vals = {
    'material' : ['PLA', 'ABS', '--', '--'], #blank values to even out table
    'lubricant' : ['None', 'LI', 'Oil', 'PTFE'],
    # 'z_sh' : [9, 8, 7][::-1],
    # 'z_r2' : [83, 82, 80][::-1],
    # 'x_s' : [-0.5, 0, 1],
    # 'x_r2' : [-0.5, 0, 0.5],
    'z_sh' : [10, 11, 15],
    'z_r2' : [70, 71, 72],
    'x_s' : [0.9 , 0.4, 0.3],
    'x_r2' : [-1.  , -0.45, -0.55],
    'Cl' : [0.4e-3, 0.3e-3, 0.2e-3][::-1], #NOTE - use 0.1 for paper, though they likely use 0.0
    'gear_thickness' : [12, 8, 6][::-1],
}

# Include reference paper values as 1 set
paper_vals = [6, 81, .5, 1, 0.1e-3, 10] # NOTE - 6 = 12 (z_s) / 2.

# Put the dictionary into a vector & count total possible designs
quick_param = []
for i in (range(2,len(param_vals))):
    key_i = list(param_vals.keys())[i]
    param_vals[key_i].insert(0, paper_vals[i-2])
    quick_param.append(param_vals[key_i])

# ALL work:
param_vals = {
    'material' : ['PLA', 'ABS', '--', '--'], #blank values to even out table
    'lubricant' : ['None', 'LI', 'Oil', 'PTFE'],
    'z_sh' : [9, 10, 11, 15],
    'z_r2' : [70, 71, 72, 73],
    'x_s' : [1, 0.9 , 0.8, 0.4],
    'x_r2' : [-0.25, -0.35  , -0.45, -0.55],
    'Cl' : [0.4e-3, 0.3e-3, 0.2e-3, 0.1e-3][::-1], #NOTE - use 0.1 for paper, though they likely use 0.0
    'gear_thickness' : [12, 10, 8, 6][::-1],
}
# Better gear ratios:
param_vals = {
    'material' : ['PLA', 'ABS', '--', '--'], #blank values to even out table
    'lubricant' : ['None', 'LI', 'Oil', 'PTFE'],
    'z_sh' : [6, 7, 8, 9],
    'z_r2' : [72, 73, 74, 75],
    'x_s' : [1.0, 0.9 , 0.8, -1],
    'x_r2' : [1, 0.9, 0.8, 0.7],
    'Cl' : [0.4e-3, 0.3e-3, 0.2e-3, 0.1e-3][::-1], #NOTE - use 0.1 for paper, though they likely use 0.0
    'gear_thickness' : [12, 10, 8, 6][::-1],
}

headers = list(param_vals.keys())
rows = list(zip(*param_vals.values()))

print(tabulate(rows, headers=headers, tablefmt="grid"))

design_vals, I1, I2, gr_s, ratios = validate_parameters(quick_param[:-1])

design_vals = np.array(design_vals)

print(f'Design vals shape: {design_vals.shape}') # Note that 8 parameters yield 9 geometric design values

+------------+-------------+--------+--------+-------+--------+--------+------------------+
| material   | lubricant   |   z_sh |   z_r2 |   x_s |   x_r2 |     Cl |   gear_thickness |
+============+=============+========+========+=======+========+========+==================+
| PLA        | None        |      6 |     72 |   1   |    1   | 0.0001 |                6 |
+------------+-------------+--------+--------+-------+--------+--------+------------------+
| ABS        | LI          |      7 |     73 |   0.9 |    0.9 | 0.0002 |                8 |
+------------+-------------+--------+--------+-------+--------+--------+------------------+
| --         | Oil         |      8 |     74 |   0.8 |    0.8 | 0.0003 |               10 |
+------------+-------------+--------+--------+-------+--------+--------+------------------+
| --         | PTFE        |      9 |     75 |  -1   |    0.7 | 0.0004 |               12 |
+------------+-------------+--------+--------+-------+--------+--------+--------

## Use L32b Taguchi array to prepare design parameters for each test

In [5]:
Taguchi = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
[1, 1, 2, 2, 2, 2, 2, 2, 2, 2],
[1, 1, 3, 3, 3, 3, 3, 3, 3, 3],
[1, 1, 4, 4, 4, 4, 4, 4, 4, 4],
[1, 2, 1, 1, 2, 2, 3, 3, 4, 4],
[1, 2, 2, 2, 1, 1, 4, 4, 3, 3],
[1, 2, 3, 3, 4, 4, 1, 1, 2, 2],
[1, 2, 4, 4, 3, 3, 2, 2, 1, 1],
[1, 3, 1, 2, 3, 4, 1, 2, 3, 4],
[1, 3, 2, 1, 4, 3, 2, 1, 4, 3],
[1, 3, 3, 4, 1, 2, 3, 4, 1, 2],
[1, 3, 4, 3, 2, 1, 4, 3, 2, 1],
[1, 4, 1, 2, 4, 3, 3, 4, 2, 1],
[1, 4, 2, 1, 3, 4, 4, 3, 1, 2],
[1, 4, 3, 4, 2, 1, 1, 2, 4, 3],
[1, 4, 4, 3, 1, 2, 2, 1, 3, 4],
[2, 1, 1, 4, 1, 4, 2, 3, 2, 3],
[2, 1, 2, 3, 2, 3, 1, 4, 1, 4],
[2, 1, 3, 2, 3, 2, 4, 1, 4, 1],
[2, 1, 4, 1, 4, 1, 3, 2, 3, 2],
[2, 2, 1, 4, 2, 3, 4, 1, 3, 2],
[2, 2, 2, 3, 1, 4, 3, 2, 4, 1],
[2, 2, 3, 2, 4, 1, 2, 3, 1, 4],
[2, 2, 4, 1, 3, 2, 1, 4, 2, 3],
[2, 3, 1, 3, 3, 1, 2, 4, 4, 2],
[2, 3, 2, 4, 4, 2, 1, 3, 3, 1],
[2, 3, 3, 1, 1, 3, 4, 2, 2, 4],
[2, 3, 4, 2, 2, 4, 3, 1, 1, 3],
[2, 4, 1, 3, 4, 2, 4, 2, 1, 3],
[2, 4, 2, 4, 3, 1, 3, 1, 2, 4],
[2, 4, 3, 1, 2, 4, 2, 4, 3, 1],
[2, 4, 4, 2, 1, 3, 1, 3, 4, 2],
])-1

In [6]:
param_cols = ['iteration','material','lubricant','z_sh', 'z_r2', 'xs', 'xr2', 'Cl', 'gear thickness', 'eta_f', 'eta_b', 'gear ratio']
# need to make a list (columns) of tuples (rows)
test_params = []
# use the Taguchi array to get the parameter values
for i in range(len(Taguchi)):
    Tag_vals = Taguchi[i][:8]
    new_params = tuple()
    for ind, val in enumerate(Tag_vals):
        key_i = list(param_vals.keys())[ind]
        new_params = new_params + (param_vals[key_i][val],)
    
    # Solve estimated efficiency:
    design_vals, _, _, _, ratio = validate_parameters(new_params[2:-1])
    eta_f, eta_b = efficiency_calc(design_vals)

    new_params = new_params + (eta_f, eta_b, ratio)
    new_params = (i+1,) + new_params
    test_params.append(new_params)   

print(tabulate(test_params, headers=param_cols, tablefmt="grid"))

# how many eta_b are 'viable' (above 0.3)?
viable_count = 0
for i in range(len(test_params)):
    if test_params[i][-2] > 0.3:
        viable_count += 1
print(f'Presumably viable designs: {viable_count} out of {len(test_params)}')

+-------------+------------+-------------+--------+--------+------+-------+--------+------------------+----------+----------+--------------+
|   iteration | material   | lubricant   |   z_sh |   z_r2 |   xs |   xr2 |     Cl |   gear thickness |    eta_f |    eta_b |   gear ratio |
+=============+============+=============+========+========+======+=======+========+==================+==========+==========+==============+
|           1 | PLA        | None        |      6 |     72 |  1   |   1   | 0.0001 |                6 | 0.652075 | 0.538141 |      44.9286 |
+-------------+------------+-------------+--------+--------+------+-------+--------+------------------+----------+----------+--------------+
|           2 | PLA        | None        |      7 |     73 |  0.9 |   0.9 | 0.0002 |                8 | 0.626198 | 0.48023  |      41.7143 |
+-------------+------------+-------------+--------+--------+------+-------+--------+------------------+----------+----------+--------------+
|           3

## Calculate design (CAD) parameters from design parameters

In [11]:
# design features:
d_cols = ['iteration', 'z_s', 'z_p1', 'z_p2', 'z_r2', 'xs', 'xp1', 'xp2', 'xr2', 'r_a [m]', 'gear_thickness']

# need to make a list (columns) of tuples (rows)
d_params = []
for j in range(len(test_params)):
    quick_param = list(test_params[j])[3:-4]
    design_vals, I1, I2, gr_s, ratios = validate_parameters(quick_param)
    design_vals = np.array(design_vals)
    design_vals[0] = design_vals[0] * 2. # z_s is 2*z_sh
    design_vals = np.vstack([design_vals,[test_params[j][-4]]])
    # add j+1 to the front of design_vals
    design_vals = np.insert(design_vals, 0, [j+1])
    new_d = tuple(design_vals)
    # for i in range(len(d_cols)):
    #     new_d = new_d + (design_vals[i])
    d_params.append(new_d)

print(tabulate(d_params, headers=d_cols, tablefmt="grid"))

+-------------+-------+--------+--------+--------+------+-------+-------------+-------+-----------+------------------+
|   iteration |   z_s |   z_p1 |   z_p2 |   z_r2 |   xs |   xp1 |         xp2 |   xr2 |   r_a [m] |   gear_thickness |
+=============+=======+========+========+========+======+=======+=============+=======+===========+==================+
|           1 |    12 |     37 |     24 |     72 |  1   | 0.375 | 0.315932    |   1   |   0.0208  |                6 |
+-------------+-------+--------+--------+--------+------+-------+-------------+-------+-----------+------------------+
|           2 |    14 |     36 |     24 |     73 |  0.9 | 0.3   | 0.172647    |   0.9 |   0.02116 |                8 |
+-------------+-------+--------+--------+--------+------+-------+-------------+-------+-----------+------------------+
|           3 |    16 |     35 |     24 |     74 |  0.8 | 0.225 | 0.029361    |   0.8 |   0.02152 |               10 |
+-------------+-------+--------+--------+-------